In [ ]:
from tools_CNN import *
# from tools_Transformer import *
# from tools_VAE import *
import pickle
import copy

In [ ]:
import os
import warnings
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
with warnings.catch_warnings():
    warnings.simplefilter("ignore")

In [ ]:
import h5py
totalnum = 100000
testnum = int(totalnum*0.2)
import random
pics_100_exp=[]

turns = 6

In [ ]:
from tensorflow.keras.utils import Sequence
import struct
int_size = 4
class BinDataset(Sequence):
    def __init__(self, file_dir, batch_size, transform=None):
        self.file_dir = file_dir
        self.batch_size = batch_size
        self.transform = transform
        self.file_list = [f for f in os.listdir(file_dir) if f.endswith('.bin')]
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.file_list) / float(self.batch_size)))

    def __getitem__(self, index):
        files = self.file_list[index * self.batch_size:(index + 1) * self.batch_size]
        X,y= self.load_batch(files)
        return X, y

    def load_batch(self, files):
        data = []
        labels = []
        for file_name in files:
            file_path = os.path.join(self.file_dir, file_name)
            with open(file_path, "rb") as f:
                combined_data = f.read()
            int_size=4
            int1_bytes = combined_data[:int_size]
            int2_bytes = combined_data[int_size:2 * int_size]
            int3_bytes = combined_data[2 * int_size:3 * int_size]
            
            int1 = struct.unpack('i', int1_bytes)[0]
            int2 = struct.unpack('i', int2_bytes)[0]
            int3 = struct.unpack('i', int3_bytes)[0]
            
            y = np.array([int1,int2,int3])+15
            compressed_data_bytes = bytes(combined_data[3*int_size:])
            compressed_data_bytes = np.unpackbits(np.frombuffer(compressed_data_bytes, dtype=np.uint8))
            compressed_data_bytes = compressed_data_bytes.astype(np.float32).reshape((1,64, 64, 64,1))
            data.append(compressed_data_bytes)  
            labels.append(y)   
            
        return np.concatenate(data),np.array(labels)

    def on_epoch_end(self):
        np.random.shuffle(self.file_list)

In [ ]:
turn = 17
turn = 2
file_dir = 'train'+str(turn)
batch_size = 256  


train_dataset_2 = BinDataset(file_dir, batch_size)

file_dir2 = 'test'+str(turn)

test_dataset_2 = BinDataset(file_dir2, batch_size)

In [ ]:
classifier = Classifier()

loss_train = []
loss_val = []



mse_loss_fn = tf.keras.losses.MeanSquaredError()
softmax_layer = tf.keras.layers.Softmax()
loss_metric = tf.keras.metrics.Mean()
loss_metric1 = tf.keras.metrics.Mean()
loss_metric2 = tf.keras.metrics.Mean()
loss_metric3 = tf.keras.metrics.Mean()

In [ ]:
from keras.utils import to_categorical
import tensorflow_addons as tfa

mse_loss_fn = tf.keras.losses.MeanSquaredError()
alpha = 0.01
epochs = 50

optimizer = tf.keras.optimizers.Adam(learning_rate=100e-5)



In [ ]:

from sklearn.metrics import r2_score
names = list(range(1,2,1))
for epoch in range(epochs):
    for step, x_batch_train in enumerate(train_dataset_2):
        x_batch = x_batch_train[0]
        rnd_batch = x_batch_train[1]
        plot_slices(8,8,64,64,x_batch[0])
        idx_tensor = [[idx for idx in range(31)] for i in range(x_batch.shape[0])]
        one_hot_y = to_categorical(rnd_batch, num_classes=31).squeeze()
        one_hot_y_list = [one_hot_y[:,0,:],one_hot_y[:,1,:],one_hot_y[:,2,:]]
        rnd_batch_T = np.transpose(rnd_batch)

        b_tmp = 0.0

        with tf.GradientTape() as tape:
            y = classifier(x_batch)
            a = tf.compat.v1.losses.softmax_cross_entropy(one_hot_y_list, y)
            b0 = tf.nn.softmax(y)
            b1 = tf.reduce_sum(b0 * idx_tensor, 2) 
            b = alpha * mse_loss_fn(rnd_batch_T,b1)
            b_tmp = mse_loss_fn(rnd_batch_T/30.0,b1/30.0)
            loss = a + b

        grads = tape.gradient(loss, classifier.trainable_weights)

        optimizer.apply_gradients(zip(grads, classifier.trainable_weights))
        loss_metric(loss)
        loss_metric1(a)
        loss_metric2(b_tmp)
        if (epoch % 1 == 0 and step==0):
            print("Start of epoch %d" % (epoch,))
            print("step %d: mean loss = %.4f ,%.4f ,%.4f" % (step, loss_metric.result(),loss_metric1.result(),loss_metric2.result()))
    for step, x_batch_train in enumerate(test_dataset_2):
        x_batch = x_batch_train[0]
        rnd_batch = x_batch_train[1]
        idx_tensor = [[idx for idx in range(31)] for i in range(x_batch.shape[0])]
        one_hot_y = to_categorical(rnd_batch, num_classes=31).squeeze()
        one_hot_y_list = [one_hot_y[:,0,:],one_hot_y[:,1,:],one_hot_y[:,2,:]]
        rnd_batch_T = np.transpose(rnd_batch)
        y = classifier(x_batch)
        a = tf.compat.v1.losses.softmax_cross_entropy(one_hot_y_list, y)
        b0 = tf.nn.softmax(y)
        b1 = tf.reduce_sum(b0 * idx_tensor, 2) 
        b = alpha * mse_loss_fn(rnd_batch_T,b1)
        b_tmp = mse_loss_fn(rnd_batch_T/30.0,b1/30.0)
        loss = a + b
        loss_metric3(b_tmp)
    loss_val.append(loss_metric3.result().numpy())
    loss_train.append(loss_metric2.result().numpy())
    print('train ',loss_metric2.result().numpy(), ' val ',loss_metric3.result().numpy())